In [1]:
import re

In [2]:
with open('LangId.train.French', 'r', encoding='UTF-8', errors='ignore') as f:
    data_french = f.read()
    print(len(data_french))


491611


In [3]:
with open('LangId.train.Italian', 'r', encoding='UTF-8', errors='ignore') as f:
    data_italian = f.read()
    print(len(data_italian))


491892


In [4]:
with open('LangId.train.English', 'r', encoding='UTF-8', errors='ignore') as f:
    data_english = f.read()
    print(len(data_english))


452144


## Problem 2: generate word based ngram

In [5]:
import collections
from itertools import product
from string import ascii_lowercase

def get_letter_ngram(data,n):
    clean_sentence = [" ".join(re.findall("[a-zA-Z]+", i)) for i in data.strip().lower().split("\n")]
    clean_data = "</s> " + " </s> ".join(clean_sentence) + " </s>"
    
    ngram = dict(collections.Counter([i + " " + j for (i,j) in \
                                      zip(clean_data.split(" ")[:-1],clean_data.split(" ")[1:])]))
    
    word_count = dict(collections.Counter(clean_data.split(" ")))

    return ngram, word_count

In [6]:
english_ngram, english_dict  = get_letter_ngram(data_english,2)

In [7]:
italian_ngram, italian_dict = get_letter_ngram(data_italian,2)

In [8]:
french_ngram, french_dict = get_letter_ngram(data_french,2)

In [9]:
ngram_dict = {"English":english_ngram, "Italian":italian_ngram, "French":french_ngram}
word_dict = {"English":english_dict, "Italian":italian_dict, "French":french_dict}

In [10]:
with open('LangId.test', 'r', encoding='UTF-8', errors='ignore') as f:
    data_test= f.read()

In [11]:
import numpy as np

def assign_letter_ngram(data,n,ngram_dict):
    result = []
    for index, sentence in enumerate(data_test.strip().split("\n")):
        sentence = ("</s> " + " ".join(re.findall("[a-zA-Z]+", sentence.lower())) + " </s>").split(" ")
        p = {i:1 for i in ngram_dict.keys()}
        for language in ngram_dict.keys():
            for i in range(0,len(sentence)- n + 1):
                cww = ngram_dict[language][" ".join(sentence[i:i+n])] if " ".join(sentence[i:i+n]) in ngram_dict[language] else 0
                cw = word_dict[language][sentence[i]] if sentence[i] in word_dict[language].keys() else 0
                p[language] += np.log( (cww+1) / (len(ngram_dict[language]) + cw))
        result.append(str(index+1) + " " + max(p, key = p.get))
    with open("wordLangId.out","w") as f:
        f.write("\n".join(result))
    return result

In [12]:
result = assign_letter_ngram(data_test,2,ngram_dict)

In [13]:
with open('LangId.sol', 'r', encoding='UTF-8', errors='ignore') as f:
    solution = f.read()
    
import pandas as pd
def check_solution(output, solution):
    col = ["predicted_" + i for i in ngram_dict.keys()]
    index = ["true_" + i for i in ngram_dict.keys()]
    compare = pd.DataFrame(columns=col,index=index)
    compare = compare.fillna(0)
    for i in range(len(output)):
        compare.at["true_" + solution[i].split(" ")[1], "predicted_" + output[i].split(" ")[1]] += 1
    return compare

check_solution(result, solution.strip().split("\n"))

,predicted_Italian,predicted_French,predicted_English
true_Italian,100,0,0
true_French,0,100,0
true_English,0,0,100
